In [2]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']

# feim = feim[feim['datetime']<='20190913_22380']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt>=1].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
may.sort_values(by='score', ascending=False, inplace=True)
feim.loc[may.index]

20190914_00401


,datetime,DT-M,score
feature,,,
512__V259-V320__diff__ProductCD-H_train,20190913_23145,2018-5,0.929229
512__V315-V67__diff__ProductCD-W_train,20190913_21174,2018-5,0.929038
605__addr1-R_emaildomain_prefix_C4__ProductCD-R_sum_train,20190913_15464,2018-5,0.928888
605__addr1-P_emaildomain_prefix_C4__ProductCD-R_sum_train,20190913_22514,2018-5,0.928888
512__V187-V259__ratio__ProductCD-C__ProductCD-C_train,20190913_21542,2018-5,0.928811
603__card4-card5_C8__ProductCD-R_std_train,20190913_22233,2018-5,0.928787
512__V187-V267__diff__ProductCD-S_train,20190913_23192,2018-5,0.928769
512__V187-V267__diff__ProductCD-S_train,20190913_23192,2018-4,0.945806
603__card4-addr1_C2__ProductCD-R_std_train,20190914_00005,2018-5,0.928735
